## 2.1 Realizácia predspracovania dát (5b).

- (A-1b) Dáta si rozdeľte na trénovaciu a testovaciu množinu podľa vami preddefinovaného pomeru. Ďalej pracujte len **s trénovacím datasetom**.


In [113]:
import pandas as pd
import numpy as np
import scipy.stats as stats
from pathlib import Path
from sklearn.model_selection import train_test_split
folder = Path("./070")

nacitame data

In [114]:
connections_df = pd.read_csv(folder / "connections.csv", delimiter="\t", parse_dates=['ts'])
devices_df = pd.read_csv(folder / "devices.csv", delimiter="\t")
processes_df = pd.read_csv(folder / "processes.csv", delimiter="\t", parse_dates=['ts'])
profiles_df = pd.read_csv(folder / "profiles.csv", delimiter="\t")

## Zlucenie dat z roznych tabuliek

vieme ze vsetky tabulky maju spolocny atribut imei, ten ale nie je unikatny takze iba na zaklade jeho nemozme tabulky spajat.

unikatny kluc podla ktoreho by sme mohli tabulky spojit je imei + datetime + mwra. Ak spojime tabulky na zaklade tychto atributov dostaneme validny merge

kedze tieto atributy najdeme len v datach connections.csv a processes.csv, budeme joinovat iba tieto tabulky

data rozdelime pomocou funkcie train_test_split z sklearn.model v pomere 80:20, random_state zaistuje opakovatelnost aby sme mohli pripadne debugovat

In [115]:
merged_data = pd.merge(connections_df, processes_df, on=['imei', 'ts', 'mwra'], how='inner')

# este rozdelime cas na konkretne atributy, aby
# sme ich pripadne mohli pouzit v ML

# merged_data['year'] = merged_data['ts'].dt.year
# merged_data['month'] = merged_data['ts'].dt.month
# merged_data['day'] = merged_data['ts'].dt.day
# merged_data['hour'] = merged_data['ts'].dt.hour
# merged_data['day_of_week'] = merged_data['ts'].dt.dayofweek

# tieto data sme sa rozhodli nepridavat (vid 2.2 A))

print(merged_data.shape)
merged_data.head()


# potom splitnut merged data na 80 % trainning a 20% testing pomocou train_test_split
merged_train_df, merged_test_df = train_test_split(merged_data, test_size=0.2, random_state=42)


(15426, 33)


In [116]:
merged_train_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12340 entries, 3784 to 7270
Data columns (total 33 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   ts                          12340 non-null  datetime64[ns]
 1   imei                        12340 non-null  int64         
 2   mwra                        12340 non-null  float64       
 3   c.android.youtube           12340 non-null  float64       
 4   c.katana                    12340 non-null  float64       
 5   c.dogalize                  12340 non-null  float64       
 6   c.android.chrome            12340 non-null  float64       
 7   c.android.gm                12340 non-null  float64       
 8   c.UCMobile.x86              12340 non-null  float64       
 9   c.raider                    12340 non-null  float64       
 10  c.android.vending           12340 non-null  float64       
 11  c.updateassist              12340 non-null  float64      

- (B-1b) Transformujte dáta na vhodný formát pre ML t.j. jedno pozorovanie musí byť opísané jedným riadkom a každý atribút musí byť v numerickom formáte (encoding). Iteratívne integrujte aj kroky v predspracovaní dát z prvej fázy (missing values, outlier detection) ako celok. 


## ako prve si v mergnutom datasete urcime spravne datove typy

In [117]:
# TODO: osetrit typy ak treba (asi nebude treba tak len napisat ze to je osetrene)

## Identifikujeme a vyriešime (odstránime) duplikáty

In [118]:
def remove_duplicates(df):
    # v datasetoch sa nevyskytol prípad, kde by bol duplikátny sĺpec, a preto tu riešime iba riadky

    if len(df.loc[df.duplicated(), :]) > 0:
        df_dupless = df.loc[~df.duplicated(), :]
    
    return df_dupless

## Identifikujeme a vyriešime chýbajúce hodnoty

In [119]:
def has_missing_values(df):
    return df.isnull().any().any()


def count_missing_values(col):
    return col.isna().sum()


def replace_missing_values(col):
    # lets calculate the amount of outliers to determine, whether to use mean or median

    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1

    # lets identify the outliers
    outliers = ((col < (Q1 - 1.5 * IQR)) | (col > (Q3 + 1.5 * IQR)))
    outliers_percentage = (outliers.sum() / col.count()) * 100

    if outliers_percentage < 5:
        return col.fillna(col.mean)
    else:
        return col.fillna(col.median)


def solve_missing_values(df):
    # if missing values < 5% => clip
    # if missing values >= 5% => replace with median
    # if missing values > 40% => drop column

    if not has_missing_values(df):
        print("there are no missing values")
        return df
    else:
        for column in df:
            total_rows = len(df)
            na_amount = count_missing_values(df[column])
            na_percentage = (na_amount/total_rows) * 100
            print(f'percentage: {column}: {na_percentage}%')

            # the missing values are making smaller amount than 5% so we clip them
            if na_percentage < 5:
                df = df.dropna(subset=[column])
            # the missing values will be replaced
            elif na_percentage >= 5 and na_percentage <= 40:
                df[column] = replace_missing_values(df[column])
            # there is too much missing values, lets drop the column
            else:
                df.drop(column, axis=1, inplace=True)
                print(f'dropped: {column}')
    return df


In [120]:
solve_missing_values(merged_data)

there are no missing values


,ts,imei,mwra,c.android.youtube,c.katana,c.dogalize,c.android.chrome,c.android.gm,c.UCMobile.x86,c.raider,...,p.katana,p.google,p.notifier,p.android.defcontainer,p.android.vending,p.inputmethod.latin,p.simulator,p.olauncher,p.browser.provider,p.gms.persistent
0,2018-05-05 10:00:00,3590433799317661057,1.0,12.57832,15.81566,11.44041,7.54471,8.14947,74.82810,98.66217,...,58.68759,46.55151,51.61852,36.47510,17.56601,87.35626,32.00305,69.51890,69.85184,21.51831
1,2018-05-05 10:01:00,3590433799317661073,0.0,14.53473,5.10559,13.23860,9.54599,12.96218,94.21770,33.36502,...,45.16131,34.43891,63.83929,72.42246,54.04621,35.68114,44.97550,84.71218,8.04137,76.74900
2,2018-05-05 10:02:00,863033069630348099,1.0,14.72129,13.76187,11.27464,16.56377,12.07414,73.70492,58.77622,...,59.91103,35.04185,32.01296,76.63106,38.55901,34.45914,86.26472,66.72859,75.20188,49.75180
3,2018-05-05 10:03:00,359043379931766007,1.0,14.17165,12.23936,8.53245,12.92046,12.97893,96.94804,65.95054,...,38.57900,38.61052,31.39066,16.12547,4.37465,90.40601,62.42150,18.49800,14.90605,14.40552
4,2018-05-05 10:04:00,8630330696303482071,1.0,14.52559,12.21164,10.68598,11.10452,11.09764,72.91827,69.47889,...,59.85793,50.84054,55.76454,48.84111,73.05974,97.77579,64.18499,78.20817,75.25798,0.55074
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15421,2018-05-13 20:57:00,8630330696303481008,1.0,17.17673,10.07652,14.13618,14.82835,15.14615,78.19613,68.83968,...,50.82987,57.03723,60.99230,68.80162,52.96995,75.72684,42.28340,51.32429,95.70857,69.73254
15422,2018-05-14 01:17:00,3590433799317662188,0.0,15.96258,10.55080,11.27594,7.88137,10.30128,85.66285,24.44919,...,60.65804,53.67259,60.48345,44.49713,0.00498,82.00950,87.56227,88.34858,89.90047,21.42181
15423,2018-05-14 01:17:00,3590433799317662188,0.0,15.96258,10.55080,11.27594,7.88137,10.30128,85.66285,24.44919,...,60.65804,53.67259,60.48345,44.49713,0.00498,82.00950,87.56227,88.34858,89.90047,21.42181
15424,2018-05-10 23:25:00,3590433799317661198,1.0,13.02198,17.49888,14.75037,13.74212,10.38747,90.19126,5.84256,...,84.35493,53.06551,57.39217,36.38532,43.43760,51.47962,31.20808,67.90144,9.79193,38.18186


In [121]:
def identify_outliers(a):
    lower = a.quantile(0.25) - 1.5 * stats.iqr(a)
    upper = a.quantile(0.75) + 1.5 * stats.iqr(a)
    
    return a[(a > upper) | (a < lower)]


def print_outliers_stat(df):
    for column in df:
        # Check if the column is numeric
        if pd.api.types.is_numeric_dtype(df[column]):
            outliers = identify_outliers(df[column])
            print(f'{column}: {len(outliers)} => {(len(outliers) * 100) / len(df[column]):.2f}%')


def remove_outliers(df):
    df_cleaned = df.copy()
    
    for column in df_cleaned.select_dtypes(include=['number']).columns:
        Q1 = df_cleaned[column].quantile(0.25)
        Q3 = df_cleaned[column].quantile(0.75)
        
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        
        df_cleaned = df_cleaned[(df_cleaned[column] >= lower_bound) & (df_cleaned[column] <= upper_bound)]
    
    return df_cleaned


def replace_outliers(df):
    df_replaced = df.copy()

    for column in df_replaced.select_dtypes(include=['number']).columns:
        Q1 = df_replaced[column].quantile(0.25)
        Q3 = df_replaced[column].quantile(0.75)
        
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR

        df_replaced[column] = np.where(df_replaced[column] > upper_bound, upper_bound, df_replaced[column])
        df_replaced[column] = np.where(df_replaced[column] < lower_bound, lower_bound, df_replaced[column])

    return df_replaced

In [122]:
# toto bude pravdepodobne do tela funkcie replace_outliers

# toto je tu kvoli warningu
processes_train['p.dogalize'] = processes_train['p.dogalize'].astype(float)


Q1 = processes_train['p.dogalize'].quantile(0.25)
Q3 = processes_train['p.dogalize'].quantile(0.75)
IQR = Q3 - Q1
lower_whisker = Q1 - 1.5 * IQR
upper_whisker = Q3 + 1.5 * IQR
median_value = processes_train['p.dogalize'].median()

processes_train.loc[(processes_train['p.dogalize'] < lower_whisker) |
                  (processes_train['p.dogalize'] > upper_whisker)] = median_value

processes_train = remove_outliers(processes_train)
print_outliers_stat(processes_train)

NameError: name 'processes_train' is not defined

## kodovanie ketegorii do numerickeho formatu
na enkodovanie kategorii by sme pouzili LabelEncoder pre atributy s malym poctom kategorii a OneHotEncoder pre viacero kategorii. Nase mergnute data su uz vsetky atributy numericke, takze toto netreba riesit v nasom pripade

In [ ]:
merged_train_df.head()

## Merged data
Na merged datach vidime ze su vsetky atributy numericke a tak ich mozme vsetky pouzit v ML

- (C-2b) Transformujte atribúty dát pre strojové učenie podľa dostupných techník minimálne: scaling (2 techniky), transformers (2 techniky) a ďalšie. Cieľom je aby ste testovali efekty a vhodne kombinovali v dátovom pipeline (od časti 2.3 a v 3. fáze). 

# import
Importujeme potrebne kniznice

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler, PowerTransformer, QuantileTransformer
from sklearn.pipeline import Pipeline

# odklad mwra
Predtym ako budeme skalovat data si odlozime nasu predikovanu hodnotu mwra a v mergnutom datasete ju dropneme

In [ ]:
saved_mwra = merged_train_df['mwra']
merged_train_with_mwra = merged_train_df
merged_train_df = merged_train_df.drop(['mwra'], axis=1)

In [ ]:
# TODO: CELE SKALOVANIE A TRANSFORM JE ZLE, TREBA SKALOVAT MERGED_TRAIN_DF
# A POTOM VYSLEDNY DF DAT DO TRANSFORMACIE A TAKTO VYTVORIT 4 KOMBINACIE A MAT 4 DATAFRAMY

# 1. technika skalovania
StandardScaler štandardizuje atribúty tak, že ich prevedie na distribúciu s priemerom 0 a štandardnou odchýlkou 1.

In [ ]:
scaler_standard = StandardScaler()
scaled_data_standard = scaler_standard.fit_transform(merged_data)
scaled_data_standard

# 2. technika skalovania
MinMaxScaler škáluje atribúty do zvoleného rozsahu, napríklad 0 až 1.

In [ ]:
scaler_minmax = MinMaxScaler()
scaled_data_minmax = scaler_minmax.fit_transform(merged_data)
scaled_data_minmax

# 1. technika transformers
PowerTransformer využíva Box-Cox alebo Yeo-Johnson transformáciu na stabilizáciu rozdelenia.

In [ ]:
power_transformer = PowerTransformer(method='yeo-johnson')
transformed_data_power = power_transformer.fit_transform(merged_data) # TODO: ze sem nepojde merged_data ale uz scalnute data
transformed_data_power

# 2.technika transformers
QuantileTransformer prevedie distribúciu dát na približne normálnu distribúciu pomocou kvantilov.

In [ ]:
quantile_transformer = QuantileTransformer(output_distribution='normal')
transformed_data_quantile = quantile_transformer.fit_transform(merged_data)
transformed_data_quantile

In [ ]:
# TODO: tu by asi bolo fajn este pre kazdy df z tych 4 co budeme mat vykreslit spolocny graf
# npr boxplot vsetkych atributov a pozriet sa ci su fajn rozlozene po transformaci a scalingu
# 
# pripadne ak bude treba tak zmenit techniku transformovania (scaling techniky su vporiadku) 

- (D-1b) Zdôvodnite Vaše voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)



# !!!!!! TODO !!!!!! TOTO ESTE SKONTROLOVAT A PREJST/PREPISAT

# Zdokumentovanie a Zdôvodnenie Rozhodnutí pri Transformácii Atribútov

Pri transformácii dát pre strojové učenie je dôležité zabezpečiť, aby atribúty mali vhodnú škálu, boli stabilné a aby ich distribúcia vyhovovala potrebám modelov. Nižšie je uvedené zdôvodnenie zvolených techník transformácie a škálovania.

## 1. Scaling (Škálovanie)
Výber techník škálovania bol motivovaný snahou o optimalizáciu výkonnosti modelov, ktoré často očakávajú normalizované alebo štandardizované vstupy.

### 1.1 StandardScaler
- **Dôvod**: StandardScaler bol zvolený, pretože štandardizuje atribúty na priemer 0 a štandardnú odchýlku 1, čo je vhodné pre algoritmy citlivé na rozsah dát, ako sú SVM, k-means alebo lineárna regresia.
- **Výhoda**: Táto metóda udržuje štatistickú vlastnosť rozdelenia dát a dobre funguje pre dáta, ktoré už majú približne normálne rozdelenie.

### 1.2 MinMaxScaler
- **Dôvod**: MinMaxScaler bol vybraný pre prípady, kde bolo nutné previesť dáta do jednotného rozsahu, napríklad 0 až 1, čo je dôležité pri metódach, ako sú neurónové siete, kde normalizácia atribútov zlepšuje konvergenciu.
- **Výhoda**: Škálovanie do konkrétneho rozsahu môže zvýšiť robustnosť algoritmu voči outlierom a zlepšiť interpretovateľnosť hodnoty každého atribútu.

## 2. Transformers (Transformácie)
Transformácie sme zvolili na stabilizáciu rozdelenia dát a odstránenie šikmosti (skewness), aby sa modely mohli lepšie zamerať na relevantné vzory.

### 2.1 PowerTransformer
- **Dôvod**: PowerTransformer bol zvolený pre svoje schopnosti stabilizovať rozdelenie dát pomocou Yeo-Johnson transformácie. Táto transformácia sa odporúča, ak sú atribúty pozitívne aj negatívne, pričom transformácia stabilizuje šikmosť a robí rozdelenie viac normálnym.
- **Výhoda**: Zlepšenie normality môže pomôcť modelom lineárneho typu a optimalizáciám, ktoré sa opierajú o normálne rozdelenie dát, ako sú regresné modely a PCA.

### 2.2 QuantileTransformer
- **Dôvod**: QuantileTransformer bol zvolený pre svoju schopnosť transformovať rozdelenie dát na normálnu distribúciu bez ohľadu na pôvodné rozdelenie, čo je užitočné pri atribútoch s výraznými odchýlkami.
- **Výhoda**: Táto metóda môže efektívne normalizovať dátové body a je vhodná pre modely citlivé na rozdelenie dát, ako sú algoritmy KNN alebo Bayesovské modely.

## 3. Kombinácia Transformácií v Pipeline
Pipeline bola zostavená na jednoduchšie testovanie a iteratívne kombinácie rôznych techník. Kombinovanie krokov zjednodušuje prácu a zabezpečuje konzistentnú predprípravu dát pre modelovanie.

- **Výhoda pipeline**: Pipeline podporuje opakovateľnosť a konzistenciu v prístupe k predspracovaniu dát. Týmto spôsobom môžeme efektívne otestovať rôzne kombinácie transformácií, vybrať najlepšie varianty a následne ich jednoducho aplikovať na nové dáta alebo testovaciu množinu.

## Celkové zdôvodnenie
Tieto voľby boli zamerané na optimalizáciu dát pre rôzne modely strojového učenia, od lineárnych metód po algoritmy citlivé na rozsah a rozdelenie dát. Skúmanie viacerých techník nám umožňuje prispôsobiť transformáciu atribútov tak, aby maximalizovala výkonnosť a presnosť modelu, čo je cieľom tejto fázy prípravy dát.


## 2.2 Výber atribútov pre strojové učenie (5b)

- (A-3b) Zistite, ktoré atribúty (features) vo vašich dátach pre ML sú informatívne k predikovanej premennej (minimálne 3 techniky s porovnaním medzi sebou). 


In [ ]:
# TODO: sem treba pridat tie 4 dataframes a na nich spravit 3 techniky identifikacie vztahu

# 1. technika Korelacna matica
orelačná matica identifikuje silu vzťahu medzi každým atribútom a cieľovou premennou. Pre číselné atribúty sa často využíva Pearsonova korelácia; vysoká hodnota (kladná alebo záporná) naznačuje informatívnosť atribútu pre predikciu.

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Vytvorenie korelačnej matice
correlation_matrix = merged_data_with_mwra.corr()

# Vizualizácia korelačnej matice pre zistenie vzťahu k cieľovej premennej mwra
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", center=0)
plt.show()

korelacna matica je velmi velka, neprehladna a tazko citatelna, preto si vyberieme top 5 najvyssich hodnot k predikovanej premennej mwra

In [ ]:
# Výber korelácie pre cieľový stĺpec 'mwra' a zoradenie podľa absolútnej hodnoty
top_5_features = correlation_matrix['mwra'].sort_values(key=abs, ascending=False).head(6)  # 'mwra' bude mať koreláciu 1 so sebou

# Odstránenie 'mwra' zo zoznamu
top_5_features = top_5_features.drop('mwra')

# Zobrazenie top 5 atribútov
print(top_5_features)

vysledok korelacnej matice mozme vidiet top 5 atributov ktore pravdepodobne vplyvaju na atribut mwra

# 2. technika Random Forest
Použitím modelu Random Forest môžeme zistiť význam atribútov podľa toho, ako často a s akým dopadom sa daný atribút používa pri delení uzlov v rozhodovacích stromoch.

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# rozdelenie dataframu na dataframe ktory obsahuje iba mwra
# a taky v ktorom su ostatne atributy
X = merged_data_with_mwra.drop('mwra', axis=1)
y = merged_data_with_mwra['mwra']

# Tréning Random Forest modelu
rf_model = RandomForestClassifier(random_state=42)
rf_model.fit(X, y)

# Zobrazenie významu atribútov
feature_importances = pd.DataFrame({
    'feature': X.columns,
    'importance': rf_model.feature_importances_
}).sort_values(by='importance', ascending=False)

# Vizualizácia významu atribútov
feature_importances.plot(kind='barh', x='feature', y='importance', legend=False)
plt.show()

Vysledok random forest mozme vidiet ze na predikciu mwra pravdepodobne vplyvaju atributy (od spodu nahor)

# 3. technika Selektivna metoda s ANOVA
SelectKBest vyberá atribúty podľa štatistických testov. Pre klasifikačné úlohy sa často používa ANOVA alebo Chi-Square (Chi2) test. Táto metóda umožňuje identifikovať atribúty s najvyššou variabilitou voči cieľovej premennej. My pouzijeme Anovu pretoze mame numericke atributy, chi by sme pouzili pri kategorickych

In [ ]:
from sklearn.feature_selection import SelectKBest, f_classif

# Použitie ANOVA pre numerické atribúty
selector = SelectKBest(score_func=f_classif, k='all')
selector.fit(X, y)

# Získanie skóre a zoradenie atribútov podľa informatívnosti
scores = pd.DataFrame({
    'feature': X.columns,
    'score': selector.scores_
}).sort_values(by='score', ascending=False)

# Vizualizácia najlepších atribútov podľa SelectKBest
scores.plot(kind='barh', x='feature', y='score', legend=False)
plt.show()

Vysledok ANOVY mozme vidiet ze na predikciu mwra pravdepodobne vplyvaju atributy (od spodu nahor)

# TS problemy
Vidime warningy ohladom konstantnych hodnotach a problem s delenim. Atributy na ktore sa libka odkazuje su rok a mesiac, takze atributy ktore sme pridali z ts. kezde je s nimi problem a uz v tomto bode si myslime ze nebudu nijak prispievat na predikovanie mwra, tak ich nebudeme na zaciatku ani pridavat do dataframu.

In [ ]:
# spájanie výsledkov do jedného DataFrame
importance_df = pd.DataFrame({
    'feature': X.columns,
    'correlation_score': correlation_matrix['mwra'].abs().drop('mwra').values,  # Korelácia s cieľovou premennou
    'random_forest_importance': rf_model.feature_importances_,  # Významnosť podľa Random Forest
    'selectkbest_score': selector.scores_  # Významnosť podľa SelectKBest
})

# Normalizácia skóre na jednotnú mierku
importance_df['correlation_score'] = importance_df['correlation_score'] / importance_df['correlation_score'].max()
importance_df['random_forest_importance'] = importance_df['random_forest_importance'] / importance_df['random_forest_importance'].max()
importance_df['selectkbest_score'] = importance_df['selectkbest_score'] / importance_df['selectkbest_score'].max()

# Vypocet dolezitosti

In [ ]:
# Priemerné skóre ako vážená dôležitosť atribútu
importance_df['average_importance'] = importance_df[['correlation_score', 'random_forest_importance', 'selectkbest_score']].mean(axis=1)

importance_df.head()

# Porovnanie vysledkov Korelacnej matice, Random forest a ANOVA testingu neformalne

todo : 

- (B-1b) Zoraďte zistené atribúty v poradí podľa dôležitosti. 


# formalne hodnotenie dolezitosti atributov

Zoradíme atribúty podľa vypočítanej priemernej dôležitosti od najdôležitejších po najmenej dôležité. Tieto atribúty predstavujú finálny výber najinformatívnejších premenných.

Ako sme uz vysie spominali, vyberieme prve tri.

In [ ]:
# Zoradenie podľa priemernej dôležitosti
importance_df = importance_df.sort_values(by='average_importance', ascending=False)

# Zobrazenie výsledkov
print(importance_df[['feature', 'average_importance']])

1. p.android.gm
2. p.android.documentsui
3. c.katana

Týmto spôsobom sme zoradili atribúty podľa ich dôležitosti pre cieľovú premennú. Atribúty s najvyšším skóre by mali byť zahrnuté v ďalšom modelovaní, pričom menej dôležité atribúty môžeme odstrániť alebo im prideliť nižšiu prioritu v ďalších analýzach.

- (C-1b) Zdôvodnite Vaše voľby/rozhodnutie pre realizáciu (t.j. zdokumentovanie)

# !!!!! TODO !!! TOTO ESTE SKONTROLOVAT ALEBO NEJAK PREPISAT VLASTNYMI SLOVAMI

# Zdokumentovanie a Zdôvodnenie Rozhodnutí pri Výbere a Zoradení Atribútov

Pri výbere a zoradení atribútov sme použili rôzne metódy, aby sme zabezpečili robustnosť výsledkov a optimalizovali model pre predikciu. Nižšie sú uvedené hlavné dôvody výberu použitých techník a prístupu k zoradeniu atribútov.

## 1. Kombinácia Viacerých Techník pre Výber Atribútov
- **Dôvod**: Každá metóda výberu atribútov má svoje špecifiká a klady, a preto kombinácia viacerých techník znižuje riziko nesprávneho výberu a zvýrazňuje atribúty, ktoré sú konzistentne dôležité naprieč rôznymi metódami.
- **Prístup**: Použili sme korelačnú maticu, ktorá poskytuje intuitívny a rýchly prehľad o vzťahoch medzi atribútmi a cieľovou premennou. Random Forest a SelectKBest sme pridali, pretože poskytujú pokročilejšie metódy hodnotenia dôležitosti, pričom využívajú silné stránky štatistických testov a stromových algoritmov.

## 2. Normalizácia a Výpočet Priemernej Dôležitosti
- **Dôvod**: Každá metóda používa rôzne mierky dôležitosti, preto sme výsledky zjednotili cez normalizáciu, aby boli porovnateľné. Výpočet priemernej dôležitosti umožňuje kombinovať výsledky z rôznych metód do jedného koherentného skóre, čo nám dáva spoľahlivý základ na zoradenie atribútov.
- **Prístup**: Normalizované skóre každého atribútu z jednotlivých metód sme použili na výpočet priemernej dôležitosti. Táto metóda zaručuje, že naše konečné poradie nie je ovplyvnené extrémami v jednej z metód, a súčasne zohľadňuje viaceré aspekty dôležitosti.

## 3. Zoradenie Atribútov na základe Priemernej Dôležitosti
- **Dôvod**: Zoradenie podľa priemernej dôležitosti umožňuje zamerať sa na najvýznamnejšie atribúty pri vytváraní modelu, čo vedie k efektívnejšiemu a presnejšiemu modelu.
- **Výhoda pre model**: Zahrnutím najdôležitejších atribútov zvyšujeme presnosť modelu a znižujeme šum v dátach. Na základe tohto poradia môžeme rozhodnúť o odstránení menej dôležitých atribútov, čím znížime riziko nadmerného fitovania a zvýšime generalizovateľnosť modelu.

## Celkové Zdôvodnenie
Tieto rozhodnutia boli navrhnuté s cieľom optimalizovať efektívnosť a interpretabilitu modelu. Použitím viacerých techník a váženého prístupu sme sa snažili zaistiť, že výber atribútov nie je založený na jednej metóde, ale zohľadňuje rozmanité prístupy. Týmto spôsobom sme schopní presnejšie identifikovať najinformatívnejšie atribúty, čo je kľúčové pre dosiahnutie kvalitných výsledkov v ďalšej fáze modelovania.

## 2.3 Replikovateľnosť predspracovania (5b)

- (A-3b) Upravte váš kód realizujúci predspracovanie trénovacej množiny tak, aby ho bolo možné bez ďalších úprav znovu použiť **na predspracovanie testovacej množiny** v kontexte strojového učenia.


In [ ]:
def preprocesing(df):
    # prebehne preprocesing dataframu
    # duplikaty -> funkcia ktora odstrani duplikaty
    # missing values -> funkcia ktora ak bude menej ako 5% tak clip, ak viac tak nahradit meanom ak viac ako 40% drop cely column
    # outliers -> 
    # scaling
    # transform
    # return vyslednych preprocessed data
    pass

- (B-2b) Využite možnosti **sklearn.pipeline**

In [ ]:
temp | anoteher_def

In [ ]:
pipe(data=merged_testing, remove_duplicates, ....)